In [137]:
import numpy as np
import pandas as pd
import networkx as nx
from pyvis.network import Network
import glob
from pathlib import Path
import os
from datetime import datetime

DATA_DIR = Path(os.getcwd()).parents[1] / "out"

%matplotlib inline

In [41]:
def get_all_tp_files(working_dir=None):
    if working_dir is None:
        working_dir = DATA_DIR
    
    return Path(working_dir).glob("tp_*")

tp_files = get_all_tp_files()
turnages_generaux = [pd.read_csv(DATA_DIR / f"tg_{i}.csv") for i in range(2010, 2020)]
turnages_partiels = []
for tp in tp_files:
    try:
        _, date = str(tp).split("_")
        date = date[:-len(".csv")]
        dt = pd.to_datetime(date, format="%Y%m%d")
        
        turnages_partiels.append((dt, pd.read_csv(DATA_DIR / tp)))
    except Exception as e:
        print(f'Error while reading {tp}: {e}')

In [49]:
all_turnages_g = pd.concat(turnages_generaux, keys=range(2010, 2020), names=['Année', 'ID'])
all_turnages_p = pd.concat([y for _, y in turnages_partiels], keys=[x for x, _ in turnages_partiels])

In [53]:
all_turnages_g[["Classement", "Ancienne thurne", "Nouvelle thurne", "Catégorie"]]

Classement  Ancienne thurne Nouvelle thurne Catégorie
Année ID                                                        
2010  0             1              NaN           UC286     A-4tJ
      1             2              NaN           UC281     A-4tJ
      2             3              NaN           UC222     A-4tJ
      3             4              NaN           UC288     A-4tJ
      4             5              NaN           UE220     A-4tJ
...               ...              ...             ...       ...
2019  453         455              NaN             NaN    0J.2AL
      454         456              NaN             NaN    0J.2AL
      455         457              NaN             NaN    0J.2AL
      456         458              NaN             NaN    0J.3AL
      457         459              NaN             NaN    0J.3AL

[4022 rows x 4 columns]

In [55]:
all_turnages_p[["Classement", "Ancienne thurne", "Nouvelle thurne"]]

Classement Ancienne thurne Nouvelle thurne
2006-08-17 0            0           MC514             ext
           1            1           MC603             ext
2007-09-17 0            0             NaN           UC273
           1            1             NaN           UR236
           2            2             NaN           UR305
...                   ...             ...             ...
2006-05-04 12          12             NaN           MB208
2007-11-22 0            0           MB607           UA426
           1            1             NaN           UE212
           2            2           MB512           UR220
           3            3           MB506           UR209

[3149 rows x 3 columns]

## On peut poser la question suivante, sur l'ensemble des turnages généraux, quelles étaient les thurnes les plus populaires ?

In [60]:
all_turnages_g["Nouvelle thurne"].value_counts()[:30]

MB107    10
MC704    10
MB306    10
MC615    10
MB409    10
MC407    10
MB704    10
MC414    10
MB208    10
MB114    10
MC614    10
MB615    10
MB607    10
MB608    10
MC608    10
MB705    10
MB413    10
MB408    10
MC413    10
MB411    10
MC412    10
MB612    10
MB108    10
MB414    10
MB706    10
MC507    10
UC360     9
UC215     9
UC298     9
UC271     9
Name: Nouvelle thurne, dtype: int64

## Puis, quels sont les personnes qui ont été le plus turnés généralement puis partiellement ?

In [92]:
for target in (all_turnages_g, all_turnages_p):
    M = max(target[~pd.isna(target["Nouvelle thurne"])]["Identifiant"].value_counts())
    print(f'Il existe une personne qui a été turné {M} fois')
    # En particulier?
    K = ((target[~pd.isna(target["Nouvelle thurne"])]["Identifiant"].value_counts()) == M).value_counts()[True] # Il y a sûrement plus propre.
    print(f'Il y a eu {K} personnes qui ont été turné {M} fois sur les 20 dernières années')

Il existe une personne qui a été turné 4 fois
Il y a eu 2 personnes qui ont été turné 4 fois sur les 20 dernières années
Il existe une personne qui a été turné 8 fois
Il y a eu 2 personnes qui ont été turné 8 fois sur les 20 dernières années


## En moyenne, combien de temps est logé un normalien ?

In [114]:
all_turnages = pd.concat([all_turnages_g, all_turnages_p]) # Fusion ultime.

# Personnes qui viennent d'acquérir une thurne, soit au TP ou au TG.
acquisitions = all_turnages[~pd.isna(all_turnages["Nouvelle thurne"]) & pd.isna(all_turnages["Ancienne thurne"])] 

anneesParPersonne = acquisitions["Identifiant"].value_counts()

print(f'50 % des normaliens sont logés au moins {anneesParPersonne.median()} ans, hors conscritude.')
print(f'En moyenne: {anneesParPersonne.mean():f} ans.')

50 % des normaliens sont logés au moins 1.0 ans, hors conscritude.
En moyenne: 1.592388 ans.


## Si on prend une session de turnage, regardons le flux des migrations.

In [154]:
def create_flow_graph(turnage_df):
    dg = nx.DiGraph()
    dg.add_nodes_from(turnage_df["Nouvelle thurne"].dropna())
    dg.add_node("Le néant")
    
    tmp_df = turnage_df[["Identifiant", "Ancienne thurne", "Nouvelle thurne"]].fillna("Le néant")
    
    for _, person in tmp_df.iterrows():
        anc_thurne = person["Ancienne thurne"]
        nouv_thurne = person["Nouvelle thurne"]
        
        dg.add_edge(anc_thurne, nouv_thurne, label=person["Identifiant"])
        
    return dg

dg = create_flow_graph(turnages_generaux[-1])
g = Network(height=800, width=920, notebook=True, heading="Flux migratoires au sein d'un turnage")
g.toggle_hide_edges_on_drag(True)
g.barnes_hut()
g.from_nx(dg)
g.show("ex.html")